In [1]:
# XGBsurv benchmark
import os
import sys
import random
import numpy as np
from numpy import savetxt
import pandas as pd
from skorch import NeuralNet
from skorch.callbacks import Callback,EarlyStopping, LRScheduler
from skorch.dataset import CVSplit, ValidSplit
import torch
from torch import nn
import torch.nn.init as init
from scipy.stats import uniform as scuniform
from scipy.stats import randint as scrandint
from scipy.stats import loguniform as scloguniform 
from sklearn.model_selection import RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.preprocessing import StandardScaler, LabelEncoder, LabelBinarizer, OneHotEncoder
#from pycox.evaluation import EvalSurv
from xgbsurv.models.utils import transform_back, transform
from xgbsurv.preprocessing.dataset_preprocessing import discretizer_df
from xgbsurv.datasets import load_metabric, load_flchain, load_rgbsg, load_support
#from xgbsurv.datasets import (load_flchain, load_rgbsg, load_support,
#load_tcga)
from xgbsurv import XGBSurv
from xgbsurv.evaluation import cindex_censored, ibs

#sys.path.append('/Users/JUSC/Documents/xgbsurv_benchmarking/deep_learning/')
from loss_functions_pytorch import AFTLoss, aft_likelihood_torch, BreslowLoss, breslow_likelihood_torch

#torch.set_default_dtype(torch.float64)

/Users/JUSC/miniconda3/envs/xgbsurv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/JUSC/Documents/xgbsurv/experiments/deep_learning


## Set Parameters
deep learning hyperparameter spaces follow pycox paper
Time-to-Event Prediction with Neural Networks and Cox Regression
page 21

In [2]:
# set parameters, put into function
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 10 # set to 50
#n_iter_cind = 200
early_stopping_rounds=10
base_score = 0.0

param_grid_breslow = {
    'estimator__module__n_layers': [1, 2, 4],
    'estimator__module__n_layers': [1, 2, 4],
    'estimator__module__num_nodes': [64, 128, 256, 512],
    'estimator__module__dropout': scuniform(0.0,0.7),
    'estimator__optimizer__weight_decay': [0.4, 0.2, 0.1, 0.05, 0.02, 0.01, 0],
    'estimator__batch_size': [64, 128, 256, 512, 1024],
    #lr not in paper because of learning rate finder
    # note: setting learning rate higher would make exp(partial_hazard) explode
    #'estimator__lr': scloguniform(0.001,0.01), # scheduler unten einbauen
    #'max_epochs':  scrandint(10,20), # corresponds to num_rounds
}

## Set MLP Model

In [3]:

class SurvivalModel(nn.Module):
    def __init__(self, n_layers, in_features, num_nodes, dropout, out_features):
        super(SurvivalModel, self).__init__()
        self.n_layers = n_layers
        self.in_features = in_features
        self.num_nodes = num_nodes
        self.dropout = dropout
        self.out_features = out_features
        model = []
        # first layer
        model.append(torch.nn.Linear(in_features, num_nodes))
        model.append(torch.nn.ReLU())
        model.append(torch.nn.Dropout(dropout))
        model.append(torch.nn.BatchNorm1d(num_nodes))

        for i in range(n_layers-1):
            model.append(torch.nn.Linear(num_nodes, num_nodes))
            #init.kaiming_normal_(model[-1].weight, nonlinearity='relu')
            model.append(torch.nn.ReLU())
            model.append(torch.nn.Dropout(dropout))
            model.append(torch.nn.BatchNorm1d(num_nodes))

        # output layer
        model.append(torch.nn.Linear(num_nodes, out_features))
    
        self.layers = nn.Sequential(*model)

        # for layer in self.layers:
        #     if isinstance(layer, nn.Linear):
        #         #nn.init.uniform_(layer.weight, a=-0.5, b=0.5)
        #         nn.init.kaiming_normal_(layer.weight)


    def forward(self, x):
        res = self.layers(x)
        #print(res)
        return res


## Set Splitting Strategy

In [4]:
# Define stratified inner k-fold cross-validation
class CustomSplit(StratifiedKFold):
    def __init__(self, n_splits=5, shuffle=True, random_state=None):
        super().__init__(n_splits=n_splits, shuffle=shuffle, random_state=random_state)

    def split(self, X, y, groups=None):
        try:
            if y.shape[1]>1:
                y = y[:,0]
        except:
            pass
        bins = np.sign(y)
        return super().split(X, bins, groups=groups)

    def get_n_splits(self, X=None, y=None, groups=None):
        return self.n_splits

outer_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)
inner_custom_cv = CustomSplit(n_splits=n_outer_splits, shuffle=True, random_state=rand_state)



## Set Scoring Function

In [5]:
# Define Scorer
def custom_scoring_function(y_true, y_pred):

        #y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, np.ndarray):
            y_pred = torch.from_numpy(y_pred)
        if isinstance(y_true, np.ndarray):
            y_true = torch.from_numpy(y_true)
        if isinstance(y_pred, pd.Series):
            y_pred = torch.tensor(y_pred.values)
        if isinstance(y_true, pd.Series):
            y_true = torch.tensor(y_true.values)
        score = aft_likelihood_torch(y_pred, y_true) 
        return score.numpy()

scoring_function = make_scorer(custom_scoring_function, greater_is_better=False)

## Set Data Loading Functions

In [6]:

# adapt this for bool case
class CustomStandardScaler(StandardScaler):
    
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super().__init__(copy=copy, with_mean=with_mean, with_std=with_std)
        
    def fit(self, X, y=None):
        # Add your own code here
        return super().fit(X, y)
    
    def transform(self, X, y=None):
        # Add your own code here
        X_transformed = super().transform(X, y)
        # Add your own code here
        return X_transformed.astype(np.float32)
    
    def fit_transform(self, X, y=None):
        # Add your own code here
        
        #to_add = X[X.columns[X.columns.isin(['sex','D'])]]
        #X = X[X.columns[~X.columns.isin(['sex','D'])]]
        #df.columns.get_loc('age')
        X_transformed = super().fit_transform(X, y)
        #to_add = to_add.values.reshape(-1, 1)

        # Horizontally stack the two arrays
        #result = np.hstack((X_transformed, to_add))
        
        # Add your own code here
        return X_transformed.astype(np.float32)

class CustomStandardScaler2(StandardScaler):
    """Just to change the datatype of bool variables."""
    def __init__(self, copy=True, with_mean=True, with_std=True):
        super().__init__(copy=copy, with_mean=with_mean, with_std=with_std)
        
    def fit(self, X, y=None):
        # Add your own code here
        return super().fit(X, y)
    
    def transform(self, X, y=None):
        # Add your own code here
        X_transformed = super().transform(X, y)
        # Add your own code here
        return X_transformed.astype(np.float32)
    
    def fit_transform(self, X, y=None):
        # Add your own code here
        
        to_return = X
        #X = X[X.columns[~X.columns.isin(['sex','D'])]]
        #df.columns.get_loc('age')
        X_transformed = super().fit_transform(X, y)
        #to_add = to_add.values.reshape(-1, 1)

        # Horizontally stack the two arrays
        #result = np.hstack((X_transformed, to_add))
        
        # Add your own code here
        return to_return.astype(np.float32)
    
#ct = CustomStandardScaler()
ct = make_column_transformer(
        (CustomStandardScaler(), make_column_selector(dtype_include=['float32'])),
        (OneHotEncoder(), make_column_selector(dtype_include=['category'])), remainder='passthrough')
data = load_metabric(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
X  = data.data
print(X.shape)
X = ct.fit_transform(X)
print(X.shape)
X.dtype

(1903, 9)
(1903, 9)


dtype('float32')

In [7]:
# recommendation D
# Adapted from https://github.com/pytorch/pytorch/issues/7068.
def seed_torch(seed=42):
    """Sets all seeds within torch and adjacent libraries.
    Args:
        seed: Random seed to be used by the seeding functions.
    Returns:
        None
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    #torch.backends.cudnn.benchmark = False
    #torch.backends.cudnn.deterministic = True
    return None

class FixSeed(Callback):
    def __init__(self, seed):
        self.seed = seed

    def initialize(self):
        seed_torch(self.seed)
        return super().initialize()

## Set Training Function

In [8]:
# set parameters, put into function
n_outer_splits = 5
n_inner_splits = 5
rand_state = 42
n_iter = 1 # # set to 50
early_stopping_rounds=10
base_score = 0.0

def train_eval(X, y, net, n_iter, filename):

        dataset_name = filename.split('_')[0]
        # add IBS later
        outer_scores = {'cindex_train_'+dataset_name:[], 'cindex_test_'+dataset_name:[]}
        best_params = {'best_params_'+dataset_name:[]}
        best_model = {'best_model_'+dataset_name:[]}
        # ct = make_column_transformer(
        #  (StandardScaler(),
        #  make_column_selector(dtype_include=float,dtype_exclude=bool)),)
        # (LabelBinarizer(),
        # make_column_selector(dtype_include=bool, dtype_exclude=float)))
        ct = make_column_transformer(
        (CustomStandardScaler(), make_column_selector(dtype_include=['float32'])),
        (CustomStandardScaler2(), make_column_selector(dtype_include=['category'])), remainder='passthrough')
        #ct = make_column_transformer(
        #(StandardScaler(), make_column_selector(dtype_include=['float32'])),
        #(OneHotEncoder(), make_column_selector(dtype_include=['category'])), remainder='passthrough')
        #columns_to_transform = [col for col in X.columns if col != 'sex']
        #ct = make_column_transformer(
        #(StandardScaler(), columns_to_transform), 
        #(LabelBinarizer(), make_column_selector(dtype_include='bool')),
        #remainder='passthrough'
        #)
        pipe = Pipeline(
        [('scaler', ct),
        ('estimator', net)]
        )
        # pipe = Pipeline([
        # ('scale', StandardScaler()),
        # ('estimator', net),
        # ])
        rs = RandomizedSearchCV(pipe, param_grid_breslow, scoring = scoring_function, n_jobs=-1, 
                            cv=inner_custom_cv, n_iter=n_iter, refit=True)
        
        for i, (train_index, test_index) in enumerate(outer_custom_cv.split(X, y)):
                # Split data into training and testing sets for outer fold
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]

                print(X_train.shape, type(X_train))
                print(y_train.shape, type(y_train))
                print(X_test.shape, type(X_test))
                print(y_test.shape, type(y_test))
                # save splits and data
                savetxt('splits/aft_train_index_'+str(i)+'_'+filename, train_index, delimiter=',')
                savetxt('splits/aft_test_index_'+str(i)+'_'+filename, test_index, delimiter=',')
                
                savetxt('splits/aft_X_train_'+str(i)+'_'+filename, X_train, delimiter=',')
                savetxt('splits/aft_X_test_'+str(i)+'_'+filename, X_test, delimiter=',')

                savetxt('splits/aft_y_train_'+str(i)+'_'+filename, y_train, delimiter=',')
                savetxt('splits/aft_y_test_'+str(i)+'_'+filename, y_test, delimiter=',')
                
                #savetxt('splits/test_index_'+str(i)+'.csv', test_index, delimiter=',')
                
                # create validation dataset for early stopping
                strat = np.sign(y_train)
                valid_split = ValidSplit(cv=0.1, stratified=strat, random_state=42)

                # train
                rs.fit(X_train, y_train)
                
                # predict
                #scaler = StandardScaler()
                #X_train = scaler.fit_transform(X_train)
                #X_test = scaler.transform(X_test)
                print(rs.best_estimator_.predict(X_train))
                best_preds_train = rs.best_estimator_.predict(X_train)
                best_preds_test = rs.best_estimator_.predict(X_test)

                #print(best_preds_test, type(best_preds_train))
                #print(best_preds_test, type(best_preds_test))
                # predict survival function
                # d = predict_survival_function(X_test, dataframe=True)

                # save predictions, get dataset name
                savetxt('predictions/train_preds_'+str(i)+'_'+filename, best_preds_train, delimiter=',')
                savetxt('predictions/test_preds_'+str(i)+'_'+filename, best_preds_test, delimiter=',')
                
                # save hyperparameter settings
                params = rs.best_estimator_.get_params
                best_params['best_params_'+dataset_name] += [rs.best_params_]
                best_model['best_model_'+dataset_name] += [params]
                #print(rs.best_params_)
                # save c-index values
                try:
                        score_train = cindex_censored(y_train, best_preds_train.reshape(-1))
                        score_test = cindex_censored(y_test, best_preds_test.reshape(-1))
                        outer_scores['cindex_train_'+dataset_name] += [score_train]
                        outer_scores['cindex_test_'+dataset_name] += [score_test]
                except:
                        outer_scores['cindex_train_'+dataset_name] += [np.nan]
                        outer_scores['cindex_test_'+dataset_name] += [np.nan]
        df_best_params = pd.DataFrame(best_params)
        df_best_model = pd.DataFrame(best_model)
        df_outer_scores = pd.DataFrame(outer_scores)
        df_metrics = pd.concat([df_best_params,df_best_model,df_outer_scores], axis=1)
        df_metrics.to_csv('metrics/aft_metric_summary_'+str(i)+'_'+filename, index=False)
        return best_model, best_params, outer_scores, best_preds_train, best_preds_test, X_train, X_test, y_train, y_test
                

In [9]:
data = load_support(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
X  = data.data
X.dtypes

age                  float32
sex                 category
race                 float32
n_comorbidities      float32
diabetes            category
dementia            category
cancer              category
blood_pressure       float32
heart_rate           float32
respiration_rate     float32
temperature          float32
white_blood_cell     float32
serum_sodium         float32
serum_creatinine     float32
dtype: object

In [10]:
data_set_fns = [load_metabric,load_flchain, load_rgbsg, load_support] #, load_flchain, load_rgbsg, load_support, load_tcga]
for dataset in data_set_fns:
    # get name of current dataset
    data = dataset(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=True)
    X  = data.data #.astype(np.float32)
    y = data.target
    print('Dataset:',data.filename)
    #print(X.dtypes)
    #print('y', y)
    
    net = NeuralNet(
    SurvivalModel, 
    module__n_layers = 1,
    module__in_features = X.shape[1],
    #module__num_nodes = 32,
    #module__dropout = 0.1, # these could also be removed
    module__out_features = 1,
    # for split sizes when result size = 1
    iterator_train__drop_last=True,
    #iterator_valid__drop_last=True,
    criterion=AFTLoss,
    optimizer=torch.optim.AdamW,
    optimizer__weight_decay = 0.4,
    batch_size=32, # separate train and valid->iterator_train__batch_size=128 and iterator_valid__batch_size=128 ?
    #callbacks=[EarlyStopping(patience=10)],
    callbacks = [
            (
                "sched",
                LRScheduler(
                    torch.optim.lr_scheduler.ReduceLROnPlateau,
                    monitor="valid_loss",
                    patience=5,
                ),
            ),
            (
                "es",
                EarlyStopping(
                    monitor="valid_loss",
                    patience=10,
                    load_best=True,
                ),
            ),
            ("seed", FixSeed(seed=42)),
        ],
    #TODO: enable stratification, verify
    train_split=ValidSplit(0.2), # might cause lower performance in metrics, explain in thesis
    lr=0.001,
    #max_epochs=1, #0,#100
    #train_split=None,
    verbose=1
    )
    best_model,params, outer_scores, best_preds_train, best_preds_test, X_train, X_test, y_train, y_test = train_eval(X, y, net, n_iter, data.filename)
    
#train eval function here


Dataset: METABRIC_adapted.csv
(1522, 9) <class 'pandas.core.frame.DataFrame'>
(1522,) <class 'pandas.core.series.Series'>
(381, 9) <class 'pandas.core.frame.DataFrame'>
(381,) <class 'pandas.core.series.Series'>
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
  epoch    train_loss    valid_loss     dur
      1        3.0600        3.2179  0.0214-------  ------------  ------------  ------
      1        3.0296        3.1985  0.0215

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.1329        3.1349  0.0216
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.1153        3.1947  0.0225
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        3.0935        3.0691  0.0226
      2        3.0246        3.1968  0.0182
      2        3.0754        3.2170  0.0193
      2        3.1015        3.1941  0.0